# Testing operations with PyTorch

In [ ]:
import numpy as np
import torch
import random
import parameters as var #Configuration and coarsening parameters
var.init() #initializes parameters
import utils as ut #Some utility functions 
import loss_function as lf #Custom loss function


from opendataset import ConfsDataset #class for opening gauge confs
from opendataset import read_binary_conf
import operators as op #Interpolator and prolongator given a set of test vectors
import operators_torch as opt #Same implementations but with pytorch

In [ ]:
def numpy_loss(pred_test_vectors,test_vectors):
    loss = 0
    for i in range(batch_size):
        ops = op.Operators(var.BLOCKS_X,var.BLOCKS_T,pred_test_vectors[i])
        for tv in range(var.NV):
            #Evaluate 
            loss += np.linalg.norm( (test_vectors[i,tv] - ops.P_Pdagg(test_vectors[i,tv])) )
            #loss = np.linealg.norm(test_vectors - P_Pdagg(test_vectors)) #|| . ||₂ (l2-norm)
    return loss

def torch_loss(pred, target):
    """
    pred  : Tensor of shape (B, NV, 2, NT, NX)   (complex numbers stored as complex dtype)
    target: Tensor of shape (B, NV, 2, NT, NX)   (the “near kernel” vectors)
    Returns a scalar loss Tensor (requires_grad=True)
    """
    batch_size = pred.shape[0]
    loss = 0.0
    for i in range(batch_size):
            
        ops = opt.Operators(var.BLOCKS_X, var.BLOCKS_T, pred[i])   
        for tv in range(var.NV):
            corrected = ops.P_Pdagg(target[i, tv])               
            diff = target[i, tv] - corrected
            loss = loss + torch.linalg.norm(diff)   
    return loss

In [ ]:
def numpy_loss_ind(pred_test_vectors,test_vectors):
    loss = 0.0
    ops = op.Operators(var.BLOCKS_X,var.BLOCKS_T,pred_test_vectors)
    if test_vectors.shape[0] == var.NV:
        for tv in range(var.NV):
            #Evaluate 
            loss += np.linalg.norm( (test_vectors[tv] - ops.P_Pdagg(test_vectors[tv])) )
            #loss = np.linealg.norm(test_vectors - P_Pdagg(test_vectors)) #|| . ||₂ (l2-norm)
    else:
        loss += np.linalg.norm( (test_vectors - ops.P_Pdagg(test_vectors)) )
    return loss

def torch_loss_ind(pred, target):
    loss = 0.0      
    ops = opt.Operators(var.BLOCKS_X, var.BLOCKS_T, pred)
    if target.shape[0] == var.NV:
        for tv in range(var.NV):
            corrected = ops.P_Pdagg(target[tv])               
            diff = target[tv] - corrected
            loss = loss + torch.linalg.norm(diff)
    else:
        loss += torch.linalg.norm( (target - ops.P_Pdagg(target)) )  
    return loss

In [ ]:
"""
Loading the configurations and the near-kernel test vectors
"""
workers = 4
batch_size = 300
dataset = ConfsDataset()
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                shuffle=False, num_workers=workers)

#----returns a tensor of size [ [batch_size,4,Nx,Nt], [batch_size,Nv,2,Nx,Nt] ]----#
first_batch = next(iter(dataloader)) 
#--------------------------------------

In [ ]:
#For testing torch vs numpy
test = first_batch[1][0]

print("set of test vectors",type(test))
oper = op.Operators(var.BLOCKS_X, var.BLOCKS_T,test)
tvecs = oper.getTestVectors()
print("numpy vectors",type(tvecs))
oper_torch = opt.Operators(var.BLOCKS_X, var.BLOCKS_T,test)
tvecs_torch = oper_torch.getTestVectors()
print("torch vectors",type(tvecs_torch))
print(tvecs.shape,tvecs_torch.shape)
print("test vectors",tvecs[0,1,2,2],tvecs_torch[0,1,2,2])

v = np.random.rand(2,var.NT,var.NX) + 1j*np.random.rand(2,var.NT,var.NX)
vc = np.random.rand(var.NV,2,var.BLOCKS_T,var.BLOCKS_X) + 1j*np.random.rand(var.NV,2,var.BLOCKS_T,var.BLOCKS_X)

vtorch = torch.tensor(v)
vctorch = torch.tensor(vc)


out = oper.P_vc(vc)
print("Pvc (np)",out[0,5,0],type(out))
outt = oper_torch.P_vc(vctorch)
print("Pvc (torch)",outt[0,5,0],type(outt))

out = oper.P_Pdagg(v)
print("PP^+ (np)",v[0,5,0],out[0,5,0])
outvc = oper.Pdagg_P(vc)
print("P^+P (np)",vc[0,0,0,0],outvc[0,0,0,0])

outt = oper_torch.P_Pdagg(vtorch)
print("PP^+ (torch)",vtorch[0,5,0],outt[0,5,0])
outvct = oper_torch.Pdagg_P(vctorch)
print("P^+P (torch)",vctorch[0,0,0,0],outvct[0,0,0,0])


In [ ]:
test_vectors = np.zeros((var.NV,2,var.NT,var.NX),dtype=complex)
#random.seed(0)
for tv in range(var.NV):
    for nt in range(var.NT):
        for nx in range(var.NX):
            for s in range(2):
                x = random.random()
                y = random.random()
                z = complex(x, y)
                test_vectors[tv,s,nt,nx] = z   #2*(nx*NT + nt) + s + 1 + tv*2*NX*NT

In [ ]:
print(numpy_loss_ind(first_batch[1][0].numpy(),first_batch[1][0].numpy()))
print(torch_loss_ind(first_batch[1][0],first_batch[1][0]))

In [ ]:
print(numpy_loss_ind(first_batch[1][0].numpy(),test_vectors))
print(torch_loss_ind(first_batch[1][0],torch.tensor(test_vectors)))

In [ ]:
loss = 0.0
remTV = 15-var.NV
for confID in range(batch_size):
    print("confID",confID)
    for i in range(remTV):
        tvector = np.zeros((2,var.NT,var.NX),dtype=complex)
        path = 'confs/near_kernel/b{0}_{1}x{2}/{3}/tvector_{1}x{2}_b{0}0000_m{4}_nconf{5}_tv{6}.tv'.format(
        int(var.BETA),var.NX,var.NT,var.M0_FOLDER,var.M0_STRING,confID,14-i)
        tvector = read_binary_conf(None,path)
        print("Test vector ",14-i)
        loss += numpy_loss_ind(first_batch[1][confID].numpy(),tvector)
        print("Numpy",numpy_loss_ind(first_batch[1][confID].numpy(),tvector))
        print("PyTorch",torch_loss_ind(first_batch[1][confID],torch.tensor(tvector)))
    print("----------------------")
print("Final loss on the remainder test vectors",loss/(batch_size*remTV))

Final loss on the remainder test vectors 9.386980374900652 (for 4 test vectors and using the rest to check the loss function)
Final loss on the remainder test vectors 2.5979268393039137 (for 14 test vectors and using the rest to check the loss function)

This shows that the more smoothed test vectors, the better they capture other near-kernel components.

Just out of curiosity. What happens if I evaluate the metric just on random vectors?

In [ ]:
loss = 0.0
for confID in range(batch_size):
    print("confID",confID)
    for i in range(var.NV):
        tvector = np.random.rand(2,var.NT,var.NX) + 1j*np.random.rand(2,var.NT,var.NX)
        print("Test vector ",14-i)
        loss += numpy_loss_ind(first_batch[1][confID].numpy(),tvector)
        print("Numpy",numpy_loss_ind(first_batch[1][confID].numpy(),tvector))
        print("PyTorch",torch_loss_ind(first_batch[1][confID],torch.tensor(tvector)))
    print("----------------------")
print("Final loss on the remainder test vectors",loss/(batch_size*var.NV))